In [2]:
sc.stop()

In [1]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="Predictor").getOrCreate()
spark = SparkSession(sc)

NameError: name 'SparkSession' is not defined

In [4]:
spark

In [5]:
df = spark.read.load("data/trends1.csv", format="csv", inferSchema="true", header="true")

In [6]:
df.show()

+----+-----+-----+-----+-------------------+
|Week|Top 1|Top 2|Top 3|               Date|
+----+-----+-----+-----+-------------------+
|   0|    4|   14|   14|2012-10-07 00:00:00|
|   1|    7|   15|   18|2012-10-14 00:00:00|
|   2|    5|   12|   15|2012-10-21 00:00:00|
|   3|    5|   15|   12|2012-10-28 00:00:00|
|   4|    4|   12|   12|2012-11-04 00:00:00|
|   5|    4|   14|   10|2012-11-11 00:00:00|
|   6|    4|   12|   12|2012-11-18 00:00:00|
|   7|    7|   14|   15|2012-11-25 00:00:00|
|   8|    4|   17|   15|2012-12-02 00:00:00|
|   9|    3|   14|   10|2012-12-09 00:00:00|
|  10|    5|    9|   11|2012-12-16 00:00:00|
|  11|    7|   13|   11|2012-12-23 00:00:00|
|  12|    6|   14|   14|2012-12-30 00:00:00|
|  13|    4|   11|   11|2013-01-06 00:00:00|
|  14|    4|   13|   16|2013-01-13 00:00:00|
|  15|    5|   16|   15|2013-01-20 00:00:00|
|  16|    4|   16|   13|2013-01-27 00:00:00|
|  17|    4|   18|   14|2013-02-03 00:00:00|
|  18|    5|   14|   15|2013-02-10 00:00:00|
|  19|    

In [7]:
df.printSchema()

root
 |-- Week: integer (nullable = true)
 |-- Top 1: integer (nullable = true)
 |-- Top 2: integer (nullable = true)
 |-- Top 3: integer (nullable = true)
 |-- Date: timestamp (nullable = true)



In [8]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
Week,261,130.0,75.48840970639135,0,260
Top 1,261,11.796934865900383,8.474903572833036,3,29
Top 2,261,36.23371647509578,26.61901744235176,9,100
Top 3,261,23.452107279693486,8.180335553818198,10,43


In [9]:
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler(inputCols = ['Week'], outputCol = 'vWeek')
vdf = vectorAssembler.transform(df)

splits = vdf.randomSplit([0.8, 0.2])
train_df = splits[0]
test_df = splits[1]

T1_train = train_df.select(["vWeek","Top 1"])
T2_train = train_df.select(["vWeek","Top 2"])
T3_train = train_df.select(["vWeek","Top 3"])

T1_test = test_df.select(["vWeek","Top 1"])
T2_test = test_df.select(["vWeek","Top 2"])
T3_test = test_df.select(["vWeek","Top 3"])

In [10]:
T1_train.show()

+------+-----+
| vWeek|Top 1|
+------+-----+
| [0.0]|    4|
| [1.0]|    7|
| [2.0]|    5|
| [3.0]|    5|
| [4.0]|    4|
| [5.0]|    4|
| [6.0]|    4|
| [7.0]|    7|
| [8.0]|    4|
|[10.0]|    5|
|[11.0]|    7|
|[12.0]|    6|
|[13.0]|    4|
|[14.0]|    4|
|[16.0]|    4|
|[17.0]|    4|
|[18.0]|    5|
|[20.0]|    3|
|[21.0]|    6|
|[22.0]|    4|
+------+-----+
only showing top 20 rows



In [11]:
from pyspark.ml.regression import LinearRegression
lr1 = LinearRegression(featuresCol = 'vWeek', labelCol='Top 1', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr2 = LinearRegression(featuresCol = 'vWeek', labelCol='Top 2', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr3 = LinearRegression(featuresCol = 'vWeek', labelCol='Top 3', maxIter=10, regParam=0.3, elasticNetParam=0.8)

lr_model1 = lr1.fit(T1_train)
lr_model2 = lr2.fit(T2_train)
lr_model3 = lr3.fit(T3_train)

print("Top 1") 
print("Coefficients: " + str(lr_model1.coefficients))
print("Intercept: " + str(lr_model1.intercept))

print("Top 2") 
print("Coefficients: " + str(lr_model2.coefficients))
print("Intercept: " + str(lr_model2.intercept))

print("Top 3") 
print("Coefficients: " + str(lr_model3.coefficients))
print("Intercept: " + str(lr_model3.intercept))

Top 1
Coefficients: [0.08845919368512574]
Intercept: 0.24514936196353063
Top 2
Coefficients: [0.29202621260780054]
Intercept: -1.546093674699278
Top 3
Coefficients: [0.08675699530450738]
Intercept: 12.352132803078526


In [12]:
trainingSummary1 = lr_model1.summary
trainingSummary2 = lr_model2.summary
trainingSummary3 = lr_model3.summary

print("Top 1")
print("RMSE: %f" % trainingSummary1.rootMeanSquaredError)
print("r2: %f" % trainingSummary1.r2)

print("Top 2")
print("RMSE: %f" % trainingSummary2.rootMeanSquaredError)
print("r2: %f" % trainingSummary2.r2)

print("Top 3")
print("RMSE: %f" % trainingSummary3.rootMeanSquaredError)
print("r2: %f" % trainingSummary3.r2)

Top 1
RMSE: 4.763272
r2: 0.685691
Top 2
RMSE: 14.712575
r2: 0.702035
Top 3
RMSE: 4.419135
r2: 0.709494


In [9]:
train_df.describe().show()

+-------+------------------+------------------+------------------+-----------------+
|summary|              Week|             Top 1|             Top 2|            Top 3|
+-------+------------------+------------------+------------------+-----------------+
|  count|               203|               203|               203|              203|
|   mean|129.64039408866995|11.665024630541872| 35.70935960591133|23.48768472906404|
| stddev| 75.58657578861401|  8.47688227144537|26.382548632763704|8.273995769827428|
|    min|                 0|                 3|                 9|               10|
|    max|               260|                29|               100|               43|
+-------+------------------+------------------+------------------+-----------------+



In [10]:
print("Top 1")
lr_predictions1 = lr_model1.transform(T1_test)
lr_predictions1.select("prediction","Top 1","vWeek").show()
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator1 = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="Top 1",metricName="r2")
print("--------------------------------------------------------------")
print("R Squared (R2) on test data = %g" % lr_evaluator1.evaluate(lr_predictions1))

test_result1 = lr_model1.evaluate(T1_test)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result1.rootMeanSquaredError)
print("--------------------------------------------------------------")
print("numIterations: %d" % trainingSummary1.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary1.objectiveHistory))
trainingSummary1.residuals.show()

Top 1
+------------------+-----+-------+
|        prediction|Top 1|  vWeek|
+------------------+-----+-------+
|0.3360659198983003|    5|  [3.0]|
|0.5149813250842847|    4|  [5.0]|
|0.6044390276772771|    4|  [6.0]|
|0.6938967302702694|    7|  [7.0]|
| 1.588473756200192|    4| [17.0]|
|1.6779314587931842|    5| [18.0]|
|2.1252199717581455|    5| [23.0]|
|2.4830507821301144|    4| [27.0]|
| 2.840881592502084|    4| [31.0]|
|3.3776278080600375|    4| [37.0]|
| 3.646000915839014|    3| [40.0]|
|3.9143740236179907|    4| [43.0]|
| 4.898408752140906|    6| [54.0]|
| 5.256239562512875|    6| [58.0]|
| 6.508647398814767|    4| [72.0]|
| 6.777020506593744|    5| [75.0]|
| 6.955935911779728|    3| [77.0]|
|  8.20834374808162|    4| [91.0]|
| 8.745089963639574|    5| [97.0]|
| 9.102920774011542|    6|[101.0]|
+------------------+-----+-------+
only showing top 20 rows

--------------------------------------------------------------
R Squared (R2) on test data = 0.69006
Root Mean Squared Error (RM

In [14]:
print("Top 2")
lr_predictions2 = lr_model2.transform(T2_test)
lr_predictions2.select("prediction","Top 2","vWeek").show()
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator2 = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="Top 2",metricName="r2")
print("--------------------------------------------------------------")
print("R Squared (R2) on test data = %g" % lr_evaluator2.evaluate(lr_predictions2))

test_result2 = lr_model2.evaluate(T2_test)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result2.rootMeanSquaredError)
print("--------------------------------------------------------------")
print("numIterations: %d" % trainingSummary2.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary2.objectiveHistory))
trainingSummary2.residuals.show()

Top 2
+------------------+-----+------+
|        prediction|Top 2| vWeek|
+------------------+-----+------+
| 1.082142238770927|   14| [9.0]|
|2.8342995144177303|   16|[15.0]|
| 4.002404364848933|   19|[19.0]|
| 5.170509215280135|   16|[23.0]|
| 7.214692703534738|   19|[30.0]|
|  7.50671891614254|   15|[31.0]|
|  8.67482376657374|   16|[35.0]|
|  9.25887619178934|   20|[37.0]|
|11.595085892651745|   17|[45.0]|
|11.887112105259547|   18|[46.0]|
|12.471164530475146|   18|[48.0]|
|15.391426656553154|   14|[58.0]|
|16.559531506984356|   14|[62.0]|
|17.143583932199956|   14|[64.0]|
| 18.60371499523896|   16|[69.0]|
| 19.47979363306236|   20|[72.0]|
|19.771819845670162|   20|[73.0]|
| 20.35587227088576|   19|[75.0]|
|20.647898483493563|   22|[76.0]|
|21.231950908709162|   19|[78.0]|
+------------------+-----+------+
only showing top 20 rows

--------------------------------------------------------------
R Squared (R2) on test data = 0.580162
Root Mean Squared Error (RMSE) on test data = 15.4

In [12]:
print("Top 3")
lr_predictions3 = lr_model3.transform(T3_test)
lr_predictions3.select("prediction","Top 3","vWeek").show()
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator3 = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="Top 3",metricName="r2")
print("--------------------------------------------------------------")
print("R Squared (R2) on test data = %g" % lr_evaluator3.evaluate(lr_predictions3))

test_result3 = lr_model3.evaluate(T3_test)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result3.rootMeanSquaredError)
print("--------------------------------------------------------------")
print("numIterations: %d" % trainingSummary3.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary3.objectiveHistory))
trainingSummary3.residuals.show()

Top 3
+------------------+-----+-------+
|        prediction|Top 3|  vWeek|
+------------------+-----+-------+
|12.141067889102008|   12|  [3.0]|
|12.320262164698109|   10|  [5.0]|
| 12.40985930249616|   12|  [6.0]|
| 12.49945644029421|   15|  [7.0]|
| 13.39542781827472|   14| [17.0]|
| 13.48502495607277|   15| [18.0]|
|13.933010645063025|   15| [23.0]|
| 14.29139919625523|   18| [27.0]|
|14.649787747447434|   14| [31.0]|
| 15.18737057423574|   13| [37.0]|
| 15.45616198762989|   14| [40.0]|
|15.724953401024043|   14| [43.0]|
|16.710521916802605|   16| [54.0]|
|17.068910467994808|   17| [58.0]|
| 18.32327039716752|   16| [72.0]|
|18.592061810561674|   16| [75.0]|
|18.771256086157774|   23| [77.0]|
| 20.02561601533049|   25| [91.0]|
|20.563198842118794|   21| [97.0]|
|20.921587393310997|   17|[101.0]|
+------------------+-----+-------+
only showing top 20 rows

--------------------------------------------------------------
R Squared (R2) on test data = 0.696685
Root Mean Squared Error (R